# 收集当当网上某类书籍的售卖信息
## 收集进入的第一页上的书籍信息，并保存为CSV格式

In [35]:
import requests
import time
import pandas as pd
from lxml import etree
#注意不要多次运行,可能反爬
def collect_book_name(bookname):
    time.sleep(5)
    test_url = 'http://search.dangdang.com/?key='+bookname
    
    content_page = requests.get(test_url).text#读取数据信息
    #print(content_page[:1000])#输出前1000个字符
    books=[]
    page = etree.HTML(content_page)
    #print(page)
    book_name = page.xpath('//li/p/a[@name="itemlist-title"]/@title')
    price_now = page.xpath('//li/p/span[@class="search_now_price"]/text()')
    comment_num = page.xpath('//li/p[@class="search_star_line"]/a[@class="search_comment_num"]/text()')
    stars = page.xpath('//li/p[@class="search_star_line"]/span/span/@style')
    #book_author = page.xpath('//li/p[@class="search_book_author"]/span/a[@name="itemlist-author"]/text()')
    pub_info = page.xpath('//li/p[@class="search_book_author"]')
    pub_info = [book_pub.xpath('string(.)') for book_pub in pub_info]
    for book in zip(book_name, pub_info, price_now, stars, comment_num):
        books.append(list(book))
    print('当当网上关于{}的信息'.format(bookname))
    print()
    
    
    books_df = pd.DataFrame(books,columns=["书名","出版信息","当前价格","星级","评论数"])
    books_df.to_csv('图书—'+bookname+'.csv',index=None,encoding="utf8")
    print('{}的首页书籍收集成功'.format(bookname))

In [36]:
collect_book_name('数学建模')

当当网上关于数学建模的信息

数学建模的首页书籍收集成功


In [37]:
#data = pd.read_csv('图书—机器学习.csv',encoding='utf-8')
data = pd.read_csv('图书—数学建模.csv',encoding='utf8')
data.head()

,书名,出版信息,当前价格,星级,评论数
0,数学建模（原书第5版，从离散建模和连续建模两部分介绍整个建模过程的原理，使读者在创造性模型...,吉奥丹诺 等著，叶其孝 等译 /2014-10-01 /机械工业出版社,¥69.30,width: 100%;,5356条评论
1,数学建模算法与应用(第2版) 参加全国数学建模竞赛、美赛的必备参考书，适合初学者的数学建模...,司守奎，孙兆亮 主编 /2015-04-01 /国防工业出版社,¥34.10,width: 90%;,6657条评论
2,"数学建模方法与分析（原书第4版,五步方法帮助读者迅速掌握数学建模的真谛）",(美) Mark M. Meerschaert著 /2015-01-01 /机械工业出版社,¥45.80,width: 100%;,2864条评论
3,MATLAB数学建模方法与实践(第3版)前后已加印20余次，MathWorks鼎力推荐。程...,卓金武 王鸿钧 编著 /2018-07-11 /北京航空航天大学出版社,¥46.60,width: 90%;,5087条评论
4,数学建模的思想和方法,张世斌 编著 /2015-03-01 /上海交通大学出版社,¥25.20,width: 100%;,134条评论


-----
-----
## 爬取多个页面的数据信息,观察网页变换，添加page_index参数

In [1]:
from lxml import etree#定义解析函数
def extract_books_from_content(content_page):
    books=[]
    page = etree.HTML(content_page)
    book_name = page.xpath('//li/p/a[@name="itemlist-title"]/@title')
    price_now = page.xpath('//li/p/span[@class="search_now_price"]/text()')
    comment_num = page.xpath('//li/p[@class="search_star_line"]/a[@class="search_comment_num"]/text()')
    stars = page.xpath('//li/p[@class="search_star_line"]/span/span/@style')
    #book_author = page.xpath('//li/p[@class="search_book_author"]/span/a[@name="itemlist-author"]/text()')
    pub_info = page.xpath('//li/p[@class="search_book_author"]')
    pub_info = [book_pub.xpath('string(.)') for book_pub in pub_info]
    for book in zip(book_name, pub_info, price_now, stars, comment_num):
        books.append(list(book))
    return books

In [5]:
books_df.to_csv('book_information.csv',index=None,encoding='utf8')#转化为csv文件

In [4]:
import requests
import time
import pandas as pd
from lxml import etree
#注意不要多次运行,可能反爬
books_total=[]
def col_books_pages(bookname,maxpage):
    time.sleep(5)
    for page in range(1,maxpage):
        test_url = 'http://search.dangdang.com/?key='+bookname+'&page_index='+str(page)
        content_page = requests.get(test_url).text#读取数据信息
        books = extract_books_from_content(content_page)#网页图书内容既解析
        books_total.extend(books)#将提取到的内容添加到原列表后面
        print("page"+str(page)+","+str(len(books))+"book download")
    print('当当网上关于{}的信息'.format(bookname))
    print()
    books_total_df = pd.DataFrame(books_total,columns=["书名","出版信息","当前价格","星级","评论数"])
    books_total_df.to_csv('图书—'+bookname+str(maxpage)+'.csv',index=None,encoding='utf8')
    time.sleep(8)
    print('信息收集完毕')

In [5]:
col_books_pages('数学建模',5)

page1,60book download
page2,59book download
page3,59book download
page4,60book download
当当网上关于数学建模的信息

信息收集完毕


In [6]:
import numpy as np
np.array(books_total).shape#每个页面作为一个维度，注意修改提取

(238, 5)

# 爬取的数据处理清洗

In [40]:
import pandas as pd
data = pd.read_csv('图书—数学建模5.csv',encoding='utf8')
data.head()

,书名,出版信息,当前价格,星级,评论数
0,数学建模（原书第5版，从离散建模和连续建模两部分介绍整个建模过程的原理，使读者在创造性模型...,吉奥丹诺 等著，叶其孝 等译 /2014-10-01 /机械工业出版社,¥71.70,width: 100%;,5386条评论
1,数学建模算法与应用(第2版) 参加全国数学建模竞赛、美赛的必备参考书，适合初学者的数学建模...,司守奎，孙兆亮 主编 /2015-04-01 /国防工业出版社,¥34.10,width: 90%;,6665条评论
2,"数学建模方法与分析（原书第4版,五步方法帮助读者迅速掌握数学建模的真谛）",(美) Mark M. Meerschaert著 /2015-01-01 /机械工业出版社,¥45.80,width: 100%;,2880条评论
3,MATLAB数学建模方法与实践(第3版)前后已加印20余次，MathWorks鼎力推荐。程...,卓金武 王鸿钧 编著 /2018-07-11 /北京航空航天大学出版社,¥46.60,width: 100%;,5090条评论
4,数学建模的思想和方法,张世斌 编著 /2015-03-01 /上海交通大学出版社,¥25.20,width: 100%;,135条评论


+ 需要做的预处理有：
    1. 去掉价格中的'￥'符号，转化为数值格式
    2. 星级转化为数字格式，取值[0:5:0.5],例如width:90%,对应$0.9*5=4.5$
    3. 对评论数直接提取数值
    4. 出版信息分为三列，分别是作者、出版日期和简介
    5. 将原始数据中的书名拆分为书名和简介两列

### 去掉价格中的'￥'符号，转化为数值格式

In [41]:
import re
def get_numbers(x):#最多匹配6个数字
    regex_num="\d{1,4}\.{0,1}\d{0,2}"#编写匹配数字的正则 表达式数字（1-4位）.(或有或无)数字（0-2位）
    return float(re.search(regex_num,x)[0])

get_numbers('￥1256.12')

1256.12

In [42]:
#提取价格￥

data['当前价格'] = data['当前价格'].map(get_numbers)

In [4]:
data['当前价格'].sample(3)

127     29.60
153     49.81
135    203.75
Name: 当前价格, dtype: float64

### 提取评论数

In [43]:
data['评论数']=data['评论数'].map(get_numbers)

In [6]:
data['评论数'].sample(3)

54     98.0
233     0.0
231     0.0
Name: 评论数, dtype: float64

In [44]:
#转为int类型
data['评论数'] = data['评论数'].astype("int")
data['评论数'].sample(3)

64     2
24     3
113    1
Name: 评论数, dtype: int32

### 转换星级

In [45]:
data['星级'] = data['星级'].map(get_numbers)/20
data.head()

,书名,出版信息,当前价格,星级,评论数
0,数学建模（原书第5版，从离散建模和连续建模两部分介绍整个建模过程的原理，使读者在创造性模型...,吉奥丹诺 等著，叶其孝 等译 /2014-10-01 /机械工业出版社,71.7,5.0,5386
1,数学建模算法与应用(第2版) 参加全国数学建模竞赛、美赛的必备参考书，适合初学者的数学建模...,司守奎，孙兆亮 主编 /2015-04-01 /国防工业出版社,34.1,4.5,6665
2,"数学建模方法与分析（原书第4版,五步方法帮助读者迅速掌握数学建模的真谛）",(美) Mark M. Meerschaert著 /2015-01-01 /机械工业出版社,45.8,5.0,2880
3,MATLAB数学建模方法与实践(第3版)前后已加印20余次，MathWorks鼎力推荐。程...,卓金武 王鸿钧 编著 /2018-07-11 /北京航空航天大学出版社,46.6,5.0,5090
4,数学建模的思想和方法,张世斌 编著 /2015-03-01 /上海交通大学出版社,25.2,5.0,135


### 获取出版社信息

In [46]:
#观察到需要提取的数据是由/划分开来
data['作者'] = data['出版信息'].map(lambda x:x.split('/')[0])

In [47]:
#出版社，利用正则表达式提取

def get_publisher(x):
    regex_pub = "/(\S{1,10}出版社)"
    pub_match_result = re.search(regex_pub,x)
    #print(pub_match_result)
    if pub_match_result !=None:
        return pub_match_result[1].strip()
    else:
        return ""

get_publisher('huin/清华大学出版社')

'清华大学出版社'

In [48]:
#在数据中新增“出版社”一列

data['出版社'] = data['出版信息'].map(get_publisher)
data['出版社'].sample(10)

43         清华大学出版社
35     中国科学技术大学出版社
161        化学工业出版社
116        高等教育出版社
210        高等教育出版社
171     哈尔滨工业大学出版社
148        化学工业出版社
119        人民邮电出版社
194        机械工业出版社
172        化学工业出版社
Name: 出版社, dtype: object

In [49]:
#提取出版日期
def get_pubdate(x):
    regex_date = '/(\d{4}-\d{2}-\d{2})'
    pubdata_match_result = re.search(regex_date,x)
    if pubdata_match_result!=None:
        return pubdata_match_result[1].strip()
    else:
        return ""

get_pubdate('司守奎，孙兆亮 主编 /2015-04-01 /国防工业出版社')

'2015-04-01'

In [50]:
#添加“出版日期”列，利用pd.to_datetime将字符串格式转化为时间格式

data['出版日期'] = pd.to_datetime(data['出版信息'].map(get_pubdate))

data['出版日期'].sample(5)

46    2020-08-01
232   2000-01-01
193   2014-08-01
40    2018-10-01
21    2020-07-01
Name: 出版日期, dtype: datetime64[ns]

In [51]:
#提取出所有信息之后，删除掉“出版信息”
del data['出版信息']
data.head()

,书名,当前价格,星级,评论数,作者,出版社,出版日期
0,数学建模（原书第5版，从离散建模和连续建模两部分介绍整个建模过程的原理，使读者在创造性模型...,71.7,5.0,5386,吉奥丹诺 等著，叶其孝 等译,机械工业出版社,2014-10-01
1,数学建模算法与应用(第2版) 参加全国数学建模竞赛、美赛的必备参考书，适合初学者的数学建模...,34.1,4.5,6665,司守奎，孙兆亮 主编,国防工业出版社,2015-04-01
2,"数学建模方法与分析（原书第4版,五步方法帮助读者迅速掌握数学建模的真谛）",45.8,5.0,2880,(美) Mark M. Meerschaert著,机械工业出版社,2015-01-01
3,MATLAB数学建模方法与实践(第3版)前后已加印20余次，MathWorks鼎力推荐。程...,46.6,5.0,5090,卓金武 王鸿钧 编著,北京航空航天大学出版社,2018-07-11
4,数学建模的思想和方法,25.2,5.0,135,张世斌 编著,上海交通大学出版社,2015-03-01


In [70]:
def get_book_name(x):
    x = x.strip()
    x = re.sub('\（.*?\)*'," ",x)
    x = re.sub('\(.*?\)*',"",x)
    x = re.sub('【.*?】-*'," ",x)
    x = re.sub('[.*]',"",x)
    return x.strip().split(" ")[0]

get_book_name("数学建模（原书第5版，从离散建模和连续建模两部分介绍整个建模过程的原理，使读者在创造性模型")

'数学建模'

In [71]:
data['书名'][1]

' 数学建模算法与应用(第2版) 参加全国数学建模竞赛、美赛的必备参考书，适合初学者的数学建模教程，每年被数万数模用户选用。'

In [72]:
data["书名称"] = data['书名'].map(get_book_name)
data['书名称'].sample(10)

163               数学建模吉奥丹诺
127                     包邮
186    数学建模-基于能力矩阵的自主学习新模式
92           UMAP数学建模案例精选3
110              数学实验与数学建模
67                数学建模简明教程
227               数学建模基础案例
30                数学建模及其应用
98                  数学建模实验
68                    数学建模
Name: 书名称, dtype: object

In [73]:
data['简介'] = data['书名'].map(lambda x:x.replace(get_book_name(x),""))
data['简介'].sample(5)

171      编著,哈尔滨工业大学出版社【正版放心购】 本店达额满减，下单满29元减1，满58元减3，...
180                    【新华自营】 【新华书店,正品保障，全国多仓就近发货，速度无忧!】
200      张世斌 编著 上海交通大学出版社 9787313124531 【正版图书，质量保证，可开...
141                                 数学建模(工业和信息化十三五规划教材) 
104      数学建模竞赛：获奖论文精选与点评 自然科学 数学 数学理论 科学与自然 zui优评卷方案...
Name: 简介, dtype: object

In [74]:
#删掉“书名”一列
data1 = data.drop(['书名'],axis=1)
data1.head()

,当前价格,星级,评论数,作者,出版社,出版日期,书名称,简介
0,71.7,5.0,5386,吉奥丹诺 等著，叶其孝 等译,机械工业出版社,2014-10-01,数学建模,（原书第5版，从离散建模和连续建模两部分介绍整个建模过程的原理，使读者在创造性模型和经验模...
1,34.1,4.5,6665,司守奎，孙兆亮 主编,国防工业出版社,2015-04-01,数学建模算法与应用第2版),数学建模算法与应用(第2版) 参加全国数学建模竞赛、美赛的必备参考书，适合初学者的数学建模...
2,45.8,5.0,2880,(美) Mark M. Meerschaert著,机械工业出版社,2015-01-01,数学建模方法与分析,"（原书第4版,五步方法帮助读者迅速掌握数学建模的真谛）"
3,46.6,5.0,5090,卓金武 王鸿钧 编著,北京航空航天大学出版社,2018-07-11,MATLAB数学建模方法与实践第3版)前后已加印20余次，MathWorks鼎力推荐。程序源...,MATLAB数学建模方法与实践(第3版)前后已加印20余次，MathWorks鼎力推荐。程...
4,25.2,5.0,135,张世斌 编著,上海交通大学出版社,2015-03-01,数学建模的思想和方法,
